# SciComp - Interactive Demo

## University of California, Berkeley
### Scientific Computing Excellence

This interactive notebook demonstrates the full capabilities of SciComp including:
- Quantum mechanics simulations
- GPU acceleration
- Machine learning physics
- Advanced visualizations

**Author**: UC Berkeley SciComp Team  
**Copyright © 2025 Meshal Alawein — All rights reserved.**

## 1. Setup and Initialization

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Berkeley color scheme
BERKELEY_BLUE = '#003262'
CALIFORNIA_GOLD = '#FDB515'

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print('SciComp initialized successfully')

## 2. Quantum Mechanics Simulations

In [ ]:
# Import Berkeley SciComp quantum modules
import sys
sys.path.append('..')

from Python.Quantum.core.quantum_states import QuantumState, BellStates, EntanglementMeasures
from Python.Quantum.core.quantum_operators import PauliOperators, QuantumGates

# Create and visualize Bell states
bell_states = {
    'Φ⁺': BellStates.phi_plus(),
    'Φ⁻': BellStates.phi_minus(),
    'Ψ⁺': BellStates.psi_plus(),
    'Ψ⁻': BellStates.psi_minus()
}

# Calculate entanglement
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Quantum Bell States Analysis', fontsize=16, color=BERKELEY_BLUE, fontweight='bold')

for idx, (name, state) in enumerate(bell_states.items()):
    ax = axes[idx // 2, idx % 2]
    
    # Visualize state vector
    amplitudes = np.abs(state.state_vector)**2
    basis_labels = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']
    
    bars = ax.bar(basis_labels, amplitudes, color=CALIFORNIA_GOLD, 
                  edgecolor=BERKELEY_BLUE, linewidth=2)
    
    # Calculate concurrence
    concurrence = EntanglementMeasures.concurrence(state)
    
    ax.set_title(f'Bell State |{name}⟩ (C = {concurrence:.3f})', 
                fontweight='bold', color=BERKELEY_BLUE)
    ax.set_ylabel('Probability', fontweight='bold')
    ax.set_ylim(0, 0.6)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Quantum Bell states analyzed successfully')

## 3. GPU Acceleration Demo

In [ ]:
from Python.gpu_acceleration.cuda_kernels import GPUAccelerator, QuantumGPU, benchmark_gpu_vs_cpu

# Initialize GPU accelerator
accelerator = GPUAccelerator()
quantum_gpu = QuantumGPU(accelerator)

# Benchmark quantum state evolution
def evolve_cpu(state, H, time):
    from scipy.linalg import expm
    U = expm(-1j * H * time)
    return U @ state

# Test data
n_qubits = 10
dim = 2**n_qubits
state = np.random.randn(dim) + 1j * np.random.randn(dim)
state = state / np.linalg.norm(state)
H = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
H = (H + H.conj().T) / 2  # Make Hermitian

# Benchmark
print(f'Benchmarking quantum evolution for {n_qubits} qubits ({dim}D Hilbert space)...')

import time
start = time.time()
result_cpu = evolve_cpu(state, H, 1.0)
cpu_time = time.time() - start

start = time.time()
result_gpu = quantum_gpu.evolve_state_gpu(state, H, 1.0, steps=10)
gpu_time = time.time() - start

if accelerator.gpu_available:
    speedup = cpu_time / gpu_time
    print(f'✅ GPU Speedup: {speedup:.2f}x')
    print(f'   CPU Time: {cpu_time:.3f}s')
    print(f'   GPU Time: {gpu_time:.3f}s')
else:
    print('⚠️  GPU not available - using CPU fallback')
    print(f'   Computation Time: {cpu_time:.3f}s')

## 4. Physics-Informed Neural Networks

In [ ]:
from Python.ml_physics.physics_informed_nn import PINNConfig, HeatEquationPINN, discover_physics_from_data

# Generate synthetic data for heat equation
x = np.linspace(0, 1, 50)
t = np.linspace(0, 0.5, 20)
X, T = np.meshgrid(x, t)

# Analytical solution (for validation)
def heat_solution(x, t, alpha=0.1):
    return np.exp(-alpha * np.pi**2 * t) * np.sin(np.pi * x)

U_true = heat_solution(X, T)

# Configure and train PINN
config = PINNConfig(
    layers=[2, 20, 20, 20, 1],  # 2 inputs (x,t), 3 hidden layers, 1 output
    activation='tanh',
    learning_rate=0.001,
    epochs=500,
    physics_weight=1.0
)

print('Training Physics-Informed Neural Network for heat equation...')
pinn = HeatEquationPINN(config, thermal_diffusivity=0.1)

# Prepare training data
x_train = X.flatten()[:, np.newaxis]
t_train = T.flatten()[:, np.newaxis]
xt_train = np.hstack([x_train, t_train])
u_train = U_true.flatten()[:, np.newaxis]

# Train PINN (if TensorFlow available)
try:
    pinn.train(xt_train[:100], u_train[:100], xt_train[100:200], verbose=False)
    print('✅ PINN trained successfully')
    
    # Visualize results
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    im1 = axes[0].imshow(U_true, aspect='auto', cmap='RdBu_r')
    axes[0].set_title('True Solution', fontweight='bold', color=BERKELEY_BLUE)
    axes[0].set_xlabel('Position')
    axes[0].set_ylabel('Time')
    plt.colorbar(im1, ax=axes[0])
    
    # Predict with PINN
    U_pred = pinn.predict(xt_train).reshape(X.shape)
    
    im2 = axes[1].imshow(U_pred, aspect='auto', cmap='RdBu_r')
    axes[1].set_title('PINN Prediction', fontweight='bold', color=BERKELEY_BLUE)
    axes[1].set_xlabel('Position')
    axes[1].set_ylabel('Time')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
except:
    print('⚠️  TensorFlow not available for PINN training')

## 5. Equation Discovery from Data

In [ ]:
# Generate data from a known dynamical system
t = np.linspace(0, 10, 100)
x = 2 * np.sin(t) + 0.5 * np.cos(2*t) + 0.1 * np.random.randn(len(t))

# Discover governing equation
discovered = discover_physics_from_data(t, x)

print('🔬 Equation Discovery Results:')
print(f"Discovered equation: {discovered['equation']}")
print(f"Active terms: {discovered['active_terms']} / {discovered['library_size']}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Original data
axes[0].plot(t, x, 'o', color=CALIFORNIA_GOLD, alpha=0.6, label='Data')
axes[0].plot(t, 2*np.sin(t) + 0.5*np.cos(2*t), '--', 
            color=BERKELEY_BLUE, linewidth=2, label='True')
axes[0].set_xlabel('Time', fontweight='bold')
axes[0].set_ylabel('Value', fontweight='bold')
axes[0].set_title('Original Data', fontweight='bold', color=BERKELEY_BLUE)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Discovered coefficients
function_names = ['1', 'x', 'x²', 'x³', 'sin(x)', 'cos(x)', 'exp(-x²)']
coeffs = discovered['coefficients'][:len(function_names)]

bars = axes[1].bar(function_names, np.abs(coeffs), 
                  color=CALIFORNIA_GOLD, edgecolor=BERKELEY_BLUE, linewidth=2)
axes[1].set_ylabel('|Coefficient|', fontweight='bold')
axes[1].set_title('Discovered Equation Terms', fontweight='bold', color=BERKELEY_BLUE)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Equation discovery completed')

## 6. Interactive Quantum Circuit Builder

In [ ]:
from ipywidgets import interact, FloatSlider, Dropdown
import matplotlib.pyplot as plt

def quantum_circuit_simulator(gate_type='Hadamard', angle=0.0):
    """Interactive quantum circuit simulator."""
    
    # Initial state |0⟩
    state = QuantumState([1, 0])
    
    # Apply selected gate
    if gate_type == 'Hadamard':
        gate = QuantumGates.H
    elif gate_type == 'Pauli-X':
        gate = PauliOperators.X
    elif gate_type == 'Pauli-Y':
        gate = PauliOperators.Y
    elif gate_type == 'Pauli-Z':
        gate = PauliOperators.Z
    elif gate_type == 'Rotation-X':
        gate = QuantumGates.Rx(angle)
    elif gate_type == 'Rotation-Y':
        gate = QuantumGates.Ry(angle)
    elif gate_type == 'Rotation-Z':
        gate = QuantumGates.Rz(angle)
    else:
        gate = np.eye(2)
    
    # Apply gate
    final_state = QuantumState(gate @ state.state_vector)
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Bloch sphere representation (simplified 2D)
    theta = np.linspace(0, 2*np.pi, 100)
    axes[0].plot(np.cos(theta), np.sin(theta), 'k-', alpha=0.3)
    
    # Plot initial state
    axes[0].arrow(0, 0, 0, 1, head_width=0.05, head_length=0.05, 
                 fc=CALIFORNIA_GOLD, ec=CALIFORNIA_GOLD, label='Initial |0⟩')
    
    # Plot final state (Bloch vector)
    alpha = final_state.state_vector[0]
    beta = final_state.state_vector[1]
    x = 2 * np.real(np.conj(alpha) * beta)
    y = 2 * np.imag(np.conj(alpha) * beta)
    z = np.abs(alpha)**2 - np.abs(beta)**2
    
    axes[0].arrow(0, 0, x, z, head_width=0.05, head_length=0.05,
                 fc=BERKELEY_BLUE, ec=BERKELEY_BLUE, label='Final state')
    
    axes[0].set_xlim(-1.5, 1.5)
    axes[0].set_ylim(-1.5, 1.5)
    axes[0].set_aspect('equal')
    axes[0].set_title('Bloch Sphere (2D)', fontweight='bold', color=BERKELEY_BLUE)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Probability distribution
    probs = np.abs(final_state.state_vector)**2
    axes[1].bar(['|0⟩', '|1⟩'], probs, color=CALIFORNIA_GOLD,
               edgecolor=BERKELEY_BLUE, linewidth=2)
    axes[1].set_ylabel('Probability', fontweight='bold')
    axes[1].set_title('Measurement Probabilities', fontweight='bold', color=BERKELEY_BLUE)
    axes[1].set_ylim(0, 1.1)
    axes[1].grid(True, alpha=0.3)
    
    # Gate matrix
    im = axes[2].imshow(np.abs(gate), cmap='viridis', vmin=0, vmax=1)
    axes[2].set_title(f'{gate_type} Gate Matrix', fontweight='bold', color=BERKELEY_BLUE)
    axes[2].set_xticks([0, 1])
    axes[2].set_yticks([0, 1])
    axes[2].set_xticklabels(['|0⟩', '|1⟩'])
    axes[2].set_yticklabels(['⟨0|', '⟨1|'])
    plt.colorbar(im, ax=axes[2])
    
    plt.tight_layout()
    plt.show()
    
    # Print state information
    print(f'Final state: [{final_state.state_vector[0]:.3f}, {final_state.state_vector[1]:.3f}]')
    print(f'Measurement probabilities: P(0) = {probs[0]:.3f}, P(1) = {probs[1]:.3f}')

# Create interactive widget
print('🎮 Interactive Quantum Circuit Simulator')
print('Select a gate and adjust rotation angle (for rotation gates):')

interact(quantum_circuit_simulator,
        gate_type=Dropdown(options=['Hadamard', 'Pauli-X', 'Pauli-Y', 'Pauli-Z',
                                   'Rotation-X', 'Rotation-Y', 'Rotation-Z'],
                          value='Hadamard', description='Gate:'),
        angle=FloatSlider(min=0, max=2*np.pi, step=0.1, value=0,
                         description='Angle:'))

## 7. Summary and Next Steps

### 🎉 Congratulations!

You've successfully explored the Berkeley SciComp Framework's capabilities:

✅ **Quantum Mechanics**: Bell states, entanglement, quantum gates  
✅ **GPU Acceleration**: High-performance quantum simulations  
✅ **Machine Learning**: Physics-informed neural networks  
✅ **Equation Discovery**: Automated physics from data  
✅ **Interactive Tools**: Real-time quantum circuit simulation  

### 📚 Next Steps

1. **Explore More Modules**: Try thermal transport, spintronics, and symbolic algebra
2. **Scale Up**: Test GPU acceleration with larger systems
3. **Custom PINNs**: Create your own physics-informed networks
4. **Contribute**: Add new features to the framework

### 🐻 Go Bears!

**University of California, Berkeley**  
Scientific Computing Excellence  
💙💛 Berkeley SciComp Framework 💙💛

In [ ]:
# Display Berkeley branding
display(HTML(f'''
<div style="background: linear-gradient(135deg, {BERKELEY_BLUE} 0%, {CALIFORNIA_GOLD} 100%); 
            padding: 20px; border-radius: 10px; text-align: center; color: white;">
    <h1 style="margin: 0;">🐻 Berkeley SciComp Framework 🐻</h1>
    <h3 style="margin: 10px 0;">University of California, Berkeley</h3>
    <p style="margin: 10px 0;">Scientific Computing Excellence Since 1868</p>
    <p style="margin: 0;"><strong>Fiat Lux - Let There Be Light</strong></p>
</div>
'''))